In [5]:

!pip install -q tensorflow tensorflow-datasets

import tensorflow as tf
import tensorflow_datasets as tfds

IMG_SIZE = 224
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

(ds_train, ds_val), ds_info = tfds.load(
    'oxford_flowers102',
    split=['train+validation', 'test'],
    as_supervised=True,
    with_info=True
)

NUM_CLASSES = ds_info.features['label'].num_classes

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = ds_train.map(preprocess, num_parallel_calls=AUTOTUNE).shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_ds = ds_val.map(preprocess, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = tf.keras.applications.MobileNetV2(
    include_top=False,
    weights='imagenet',
    input_tensor=inputs
)
base_model.trainable = False
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(train_ds, validation_data=val_ds, epochs=10)

model.save("flower_model.keras")


/tmp/ipython-input-5-1134163560.py:28: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(


Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 356s 5s/step - accuracy: 0.1166 - loss: 4.4023 - val_accuracy: 0.5464 - val_loss: 2.3047
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 381s 5s/step - accuracy: 0.7162 - loss: 1.6098 - val_accuracy: 0.7268 - val_loss: 1.4603
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 347s 5s/step - accuracy: 0.8646 - loss: 0.8866 - val_accuracy: 0.7621 - val_loss: 1.1875
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 383s 5s/step - accuracy: 0.9345 - loss: 0.5593 - val_accuracy: 0.7920 - val_loss: 0.9958
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 445s 6s/step - accuracy: 0.9649 - loss: 0.3937 - val_accuracy: 0.8084 - val_loss: 0.8976
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 373s 5s/step - accuracy: 0.9806 - loss: 0.2874 - val_accuracy: 0.8148 - val_loss: 0.8316
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 331s 5s/step - accuracy: 0.9894 - loss: 0.2191 - val_accuracy: 0.8180 - val_loss: 0.8094
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 399s 5s/step - accuracy: 0.9919 - loss: 0.1772 - val_accuracy: 0.8214 - v

In [4]:

model.save("flower_model.keras")

with open("labels.json", "w") as f:
    json.dump({str(i): name.lower() for i, name in enumerate(label_names)}, f)
    from google.colab import files
files.download("flower_model.keras")
files.download("labels.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>